## <font color='darkblue'>Preface</font>
A code snippet explains thousands of words. Here we are going to use a few examples to explain topics covering Python packages [**threading**](https://docs.python.org/3/library/threading.html) and [**asyncio**](https://docs.python.org/3/library/asyncio.html). We won't go into details such as specific user cases or delicate API usages. On the contrary, a simple life situation will be used as a metaphor to explain the usage of those two similar concepts.

Now let's consider the situation that you have below tasks to handle:
* Cook breakfast (2s)
* Wash clothes (5s)
* Do house chores (3s)
* Take out the trash (1s)

Those tasks are defined below:

In [58]:
from codetiming import Timer
import concurrent.futures
import logging
import threading
import time
import queue


format = "%(asctime)s: %(message)s"
logging.basicConfig(
    format=format, level=logging.INFO, datefmt="%H:%M:%S")

class Task:
  def __init__(self, task_name: str, time_sec: int):
    self._name = task_name
    self._time_sec = time_sec
    
  @property
  def name(self):
    return self._name

  def do(self):
    logging.info('Start task %s', self._name)
    time.sleep(self._time_sec)
    logging.info('End task %s', self._name)
    
    
cook_breakfast_task = Task('Cook breakfast', 2)
wash_clothes_task = Task('Wash clothes', 5)
do_house_chores_task = Task('Wash clothes', 3)
take_output_the_trash_task = Task('Wash clothes', 1)
all_tasks = [
  cook_breakfast_task, wash_clothes_task,
  do_house_chores_task, take_output_the_trash_task]

<a id='sect0'></a>
### <font color='darkgreen'>Agenda</font>
* <font size='3ptx'><b><a href='#sect1'>Synchronous programming</a></b></font>
* <font size='3ptx'><b><a href='#sect2'>Multi-threads programming</a></b></font>
* <font size='3ptx'><b><a href='#sect3'>Async Programming</a></b></font>

<a id='sect1'></a>
## <font color='darkblue'>Synchronous programming</font> ([back](#sect0))
If you are alone and do them one by one, how much time do you need to complete all tasks? Let's find out below:

In [48]:
st = time.perf_counter()

# Iterate the tasks and handle them one by one
for task in all_tasks:
  task.do()
  
print(f'{time.perf_counter() - st:.01f}s')

20:26:40: Start task Cook breakfast
20:26:42: End task Cook breakfast
20:26:42: Start task Wash clothes
20:26:47: End task Wash clothes
20:26:47: Start task Wash clothes
20:26:50: End task Wash clothes
20:26:50: Start task Wash clothes
20:26:51: End task Wash clothes


11.0s


No doubt. It took 11 seconds!

<a id='sect2'></a>
## <font color='darkblue'>Multi-threads programming</font> ([back](#sect0))
Think about if we could have more than one person to do the task. How could we achieve that? In Python, package [**threading**](https://docs.python.org/3/library/threading.html#) is one solution for that goal. There are a few ways to implement multi-threads programming. Here we will take a look at two common practices: 

### <font color='darkgreen'>threading.Thread</font>
The [**Thread**](https://docs.python.org/3/library/threading.html#thread-objects) class represents an activity that is to run in a separate thread of control. Here we define two threads (people) to do the tasks:

In [55]:
# 0) Put tasks into the queue
task_queues = queue.Queue()
_ = [task_queues.put(task) for task in all_tasks]

# 1) Define function to tacke tasaks
def handle_tasks(worker_name: str, task_queues: queue.Queue):
  logging.info('%s start working', worker_name)
  while task_queues:
    try:
      next_task = task_queues.get(False)
      next_task.do()
    except Exception as ex:
      # When task_queues is empty, Empty exception will be thrown.
      break
    
  logging.info('%s stop working', worker_name)

In [56]:
# 2) Initialize and start two threads to handle the tasks
with Timer(text="\nTotal elapsed time: {:.1f}s"):
  workers = []
  for i in range(2):
    worker = threading.Thread(target=handle_tasks, args=(f'worker-{i+1}', task_queues))
    worker.start()
    workers.append(worker)
  
  _ = [worker.join() for worker in workers]

20:34:40: worker-1 start working
20:34:40: Start task Cook breakfast
20:34:40: worker-2 start working
20:34:40: Start task Wash clothes
20:34:42: End task Cook breakfast
20:34:42: Start task Wash clothes
20:34:45: End task Wash clothes
20:34:45: End task Wash clothes
20:34:45: Start task Wash clothes
20:34:45: worker-1 stop working
20:34:46: End task Wash clothes
20:34:46: worker-2 stop working



Total elapsed time: 6.0s


With two people to do the tasks, we could reduce the time spent from 11s to 6s. Isn't it amazing! If we have more workers, could we reduce the time more? 

In [53]:
# Put tasks into the queue
task_queues = queue.Queue()
_ = [task_queues.put(task) for task in all_tasks]

with Timer(text="\nTotal elapsed time: {:.1f}s"):
  workers = []
  for i in range(3):  # Here we have three threads to handle the tasks
    worker = threading.Thread(target=handle_tasks, args=(f'worker-{i+1}', task_queues))
    worker.start()
    workers.append(worker)
  
  # We call .join to wait for all threads to complete before moving to next line of code.
  _ = [worker.join() for worker in workers]

20:33:51: worker-1 start working
20:33:51: worker-2 start working
20:33:51: Start task Cook breakfast
20:33:51: worker-3 start working
20:33:51: Start task Wash clothes
20:33:51: Start task Wash clothes
20:33:53: End task Cook breakfast
20:33:53: Start task Wash clothes
20:33:54: End task Wash clothes
20:33:54: worker-3 stop working
20:33:54: End task Wash clothes
20:33:54: worker-1 stop working
20:33:56: End task Wash clothes
20:33:56: worker-2 stop working



Total elapsed time: 5.0s


Yes, we could reduce the time slightly from 6s to 5s. No more because that's limitation. (Thinking about we do all tasks at same time, the minimum time we need is `max(task time1, task time2, ...) = max(5, 3, 2, 1) = 5`)

### <font color='darkgreen'>ThreadPoolExecutor</font>
Another convenient way to define multiple threads is by using [**ThreadPoolExecutor**](https://docs.python.org/3/library/concurrent.futures.html#threadpoolexecutor) in package [**concurrent.futures**](https://docs.python.org/3/library/concurrent.futures.html#):

In [57]:
# Put tasks into the queue
task_queues = queue.Queue()
_ = [task_queues.put(task) for task in all_tasks]
num_worker = 3

with Timer(text="\nTotal elapsed time: {:.1f}s"):
  with concurrent.futures.ThreadPoolExecutor(max_workers=num_worker) as executor:
    for i in range(num_worker):
      executor.submit(handle_tasks, f'worker-{i}', task_queues)

20:37:51: worker-0 start working
20:37:51: Start task Cook breakfast
20:37:51: worker-1 start working
20:37:51: worker-2 start working
20:37:51: Start task Wash clothes
20:37:51: Start task Wash clothes
20:37:53: End task Cook breakfast
20:37:53: Start task Wash clothes
20:37:54: End task Wash clothes
20:37:54: End task Wash clothes
20:37:54: worker-0 stop working
20:37:54: worker-2 stop working
20:37:56: End task Wash clothes
20:37:56: worker-1 stop working



Total elapsed time: 5.0s


<a id='sect3'></a>
## <font color='darkblue'>Async Programming</font> ([back](#sect0))
What if we don't need to wait for the task to be done and can do other tasks simultaneously while waiting?  Then the Python package  [**asyncio**](https://docs.python.org/3/library/asyncio.html) could come into handy in this situation. One thing to pay attention here is that [**asyncio**](https://docs.python.org/3/library/asyncio.html) is running in only one thread.

In [43]:
import asyncio

In order for our task class to work in package [**asyncio**](https://docs.python.org/3/library/asyncio.html), we have to refactor it and make method `do` async:
```python
class AsyncTask:
  def __init__(self, task_name: str, time_sec: int):
    self._name = task_name
    self._time_sec = time_sec

  async def do(self):  # Prefix `async` is required to work in package asyncio
    logging.info('Start task %s', self._name)
    await asyncio.sleep(self._time_sec)  # await is the blocking code symbol in package asyncio
    logging.info('End task %s', self._name)
    
    
cook_breakfast_task = AsyncTask('Cook breakfast', 2)
wash_clothes_task = AsyncTask('Wash clothes', 5)
do_house_chores_task = AsyncTask('Wash clothes', 3)
take_output_the_trash_task = AsyncTask('Wash clothes', 1)
all_async_tasks = [
    cook_breakfast_task,
    wash_clothes_task,
    do_house_chores_task,
    take_output_the_trash_task]

```

Here we add prefix `async` in front of method `do` and replace `time.sleep` with `asyncio.sleep`. 

Same actions are required to refactor the function `handle_tasks`:
```python
async def handle_tasks(task_queue, name):
  timer = Timer(text=f"Task {name} elapsed time: {{:.1f}}")
  while not task_queue.empty():
    try:
      task = task_queue.get()
    except:
      break
    timer.start()
    await task.do()
    timer.stop()
```

Finally, we have to trigger the execution of async code explicitly. Check codes below:
```python
async def main():
  """
  This is the main entry point for the program
  """

  task_queue = queue.Queue()
  _ = [task_queue.put(task) for task in all_async_tasks]

  # Run the tasks
  with Timer(text="\nTotal elapsed time: {:.1f}"):
    await asyncio.gather(
        handle_tasks(task_queue, 'a1'),
        handle_tasks(task_queue, 'a2'),
        handle_tasks(task_queue, 'a3'),
    )


if __name__ == "__main__":
    asyncio.run(main())  # Trigger the execution of async code
```

For the full code implementation, please refer to `async_tasks_ex1.py`. Below is the execution example:
```shell
$ python async_tasks_ex1.py
20:16:26: Start task Cook breakfast
20:16:26: Start task Wash clothes
20:16:26: Start task Wash clothes
20:16:28: End task Cook breakfast
Task a1 elapsed time: 2.0
20:16:28: Start task Wash clothes
20:16:29: End task Wash clothes
Task a3 elapsed time: 3.0
20:16:29: End task Wash clothes
Task a1 elapsed time: 1.0
20:16:31: End task Wash clothes
Task a2 elapsed time: 5.0

Total elapsed time: 5.0
```

## <font color='darkblue'>Supplement</font>
* [Youtube - Python Threading Explained in 8 Minutes](https://www.youtube.com/watch?v=A_Z1lgZLSNc&list=PLyQnbMWK6HUUXvb9HgY_1kJfnWZw28H84&index=1)
* [RealPython - An Intro to Threading in Python](https://realpython.com/intro-to-python-threading/)
* [RealPython - Getting Started With Async Features in Python](https://realpython.com/python-async-features/)